In [1]:
import sys
sys.path.append('../../pyutils')

import numpy as np
import scipy.linalg

import metrics

np.random.seed(12)

# LU Decomposition

Let $A$ matrix of size $n * n$. Some (not all) non-singular (invertible) matrices can be decomposed with the LU Decomposition.  
$A = LU$ with $L$ lower unit triangular matrix (1s on diagonal), and $U$ upper unit triangular matrix.

In [2]:
def lu(a):
    n = a.shape[0]
    l = np.eye(n)
    u = a.copy()
    
    for j in range(n):
        pivot = u[j, j] #might be null, decomposition fails if no perms
        
        for i in range(j+1, n):
            #coeff is selected to put 0 in u[i, j]
            coeff = - u[i, j] / pivot 
            l[i, j] = - coeff
            u[i] += coeff * u[j]  
    
    return l, u

In [3]:
A = np.random.randn(4, 4)
l, u = lu(A)
print(metrics.is_ltri(l))
print(metrics.is_utri(u))
print(metrics.tdist(A, l @ u))
print(l)
print(u)

True
True
8.760130269597858e-16
[[ 1.          0.          0.          0.        ]
 [ 1.59231585  1.          0.          0.        ]
 [-1.70614387 -3.80097752  1.          0.        ]
 [ 2.31710137 -0.80894676 -0.28949308  1.        ]]
[[ 0.47298583 -0.68142588  0.2424395  -1.70073563]
 [ 0.         -0.44967612 -0.38091317  2.58788063]
 [ 0.          0.         -1.63202867  7.40723341]
 [ 0.          0.          0.          8.05642753]]


## Solve systems of equations

Let $A$ matrix of size $n * n$, $b$ and $x$ vectors of size $n$.  
Find $x$ such that $Ax = b$.  
Let $A = LU$.  
$$Ax = b$$
$$LUx = b$$
Let $c = Ux$.  
Solve $Lc = b$ with forward subtition.  
Solve $Ux = c$ with backward subtition.

In [4]:
def solve_lu(A, b):
    L, U = lu(A)
    c = scipy.linalg.solve_triangular(L, b, lower=True)
    x = scipy.linalg.solve_triangular(U, c)
    return x

In [5]:
A = np.random.randn(4, 4)
b = np.random.randn(4)

x = solve_lu(A, b)
x_sol = scipy.linalg.solve(A, b)

print(x)
print(x_sol)
print(metrics.tdist(x, x_sol))
print(metrics.tdist(A @ x, b))

[-0.04612211  0.66400747 -0.76361173  0.06805269]
[-0.04612211  0.66400747 -0.76361173  0.06805269]
4.072131729349396e-16
2.435541875787129e-16


The same algorithm also works to solve matrix systems.
Solve $AX = B$ with $A$ matrix of size $n * n$, $B$ matrix of size $n * p$, and $X$ matrix of size $n * p$

In [6]:
A = np.random.randn(4, 4)
b = np.random.randn(4, 3)

x = solve_lu(A, b)
x_sol = scipy.linalg.solve(A, b)

print(x)
print(x_sol)
print(metrics.tdist(x, x_sol))
print(metrics.tdist(A @ x, b))

[[ 0.16198301 -0.44528304 -0.41340957]
 [ 1.07189255 -1.77532918 -0.93435048]
 [ 0.03739459 -0.06904685 -0.80474698]
 [ 0.32868399 -0.5980154   0.07345963]]
[[ 0.16198301 -0.44528304 -0.41340957]
 [ 1.07189255 -1.77532918 -0.93435048]
 [ 0.03739459 -0.06904685 -0.80474698]
 [ 0.32868399 -0.5980154   0.07345963]]
2.391654643720554e-16
4.901138768834219e-16


It's possible to find the inverse of a matrix by solving $AX = B$ for $B = I$.  
$X = A^{-1}$

In [7]:
def inv_lu(A):
    return solve_lu(A, np.eye(A.shape[0]))

In [8]:
A = np.random.randn(4, 4)
Ai = inv_lu(A)
Ai_sol = np.linalg.inv(A)

print(Ai)
print(Ai_sol)
print(metrics.tdist(Ai, Ai_sol))
print(metrics.tdist(A @ Ai, np.eye(4)))
print(metrics.tdist(Ai @ A, np.eye(4)))

[[ 0.13813637 -0.6297848   0.32532762  0.58361483]
 [ 0.02442446 -0.0461876  -0.42456221 -1.14916703]
 [-0.41320576  0.06033226 -0.00953712 -0.67958276]
 [ 0.27784622  0.4398829  -0.09806117 -1.07621781]]
[[ 0.13813637 -0.6297848   0.32532762  0.58361483]
 [ 0.02442446 -0.0461876  -0.42456221 -1.14916703]
 [-0.41320576  0.06033226 -0.00953712 -0.67958276]
 [ 0.27784622  0.4398829  -0.09806117 -1.07621781]]
7.307614010947559e-16
1.1566293428875008e-15
8.65920581238202e-16


## PLU Decomposition

Any non-singular matrix $A$ of size $n * n$ can be decomposed as:
$$PA = LU$$
$L$ lower unit triangular matrix (1s on diagonal), $U$ upper unit triangular matrix, and $P$ transposition matrix.   
$P$ is used to exchange rows of $A$, in order to remove 0-pivots, that makes the $LU$ decomposition ompossible, and also to choose the biggest pivot, to have a more stable matrix.

In [9]:
def find_p(a):
    n = a.shape[0]
    p = np.eye(n)
    
    for j in range(n):
        
        #get index of biggest abs element in column j (starting at line j)
        pivot = j + np.argmax(np.abs(a[j:, j]))
        
        if pivot != j: #swap both lines
            p[[j, pivot]] = p[[pivot, j]]
            
    return p
    

def plu(a):
    p = find_p(a)
    a2 = p @ a
    l, u = lu(a2)
    return p, l, u

In [10]:
A = np.random.randn(4, 4)
p, l, u = plu(A)
print(metrics.is_ltri(l))
print(metrics.is_utri(u))
print(metrics.tdist(p @ A, l @ u))
print(p)
print(l)
print(u)

True
True
3.0531133177191805e-16
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 1.          0.          0.          0.        ]
 [-0.73237864  1.          0.          0.        ]
 [-0.25500099 -0.04167413  1.          0.        ]
 [-0.37667918 -0.17565632  0.8168222   1.        ]]
[[ 1.82882072e+00 -1.00100155e+00 -2.09169121e+00  1.46559706e-01]
 [ 0.00000000e+00 -2.42276507e+00 -1.73123730e+00  3.65109783e-01]
 [ 5.55111512e-17  0.00000000e+00 -1.00341088e+00 -1.20663501e+00]
 [-4.53427405e-17  0.00000000e+00  0.00000000e+00  1.35769598e-01]]


$PLU$ decomposition can be used to solve $Ax=b$  
Let $PA = LU$.  
$$Ax = b$$
$$PAx = Pb$$
$$LUx = Pb$$
Let $c = Ux$.  
Solve $Lc = Pb$ with forward subtition.  
Solve $Ux = c$ with backward subtition.  
Similar techniques can be used to solve matrix systems, and to find the inverse of any singular matrix.

In [11]:
def solve_plu(A, b):
    P, L, U = plu(A)
    c = scipy.linalg.solve_triangular(L, P @ b, lower=True)
    x = scipy.linalg.solve_triangular(U, c)
    return x

In [12]:
A = np.random.randn(4, 4)
b = np.random.randn(4, 3)

x = solve_plu(A, b)
x_sol = scipy.linalg.solve(A, b)

print(x)
print(x_sol)
print(metrics.tdist(x, x_sol))
print(metrics.tdist(A @ x, b))

[[ 0.67963185  0.76083933 -0.22913294]
 [-0.34468008  0.13057644 -0.47866154]
 [ 0.069055   -0.61554658 -0.31966445]
 [ 1.57513242  2.15868724 -1.07682005]]
[[ 0.67963185  0.76083933 -0.22913294]
 [-0.34468008  0.13057644 -0.47866154]
 [ 0.069055   -0.61554658 -0.31966445]
 [ 1.57513242  2.15868724 -1.07682005]]
0.0
1.0812435546590148e-15


In [13]:
def inv_plu(A):
    return solve_plu(A, np.eye(A.shape[0]))

In [14]:
A = np.random.randn(4, 4)
Ai = inv_plu(A)
Ai_sol = np.linalg.inv(A)

print(Ai)
print(Ai_sol)
print(metrics.tdist(Ai, Ai_sol))
print(metrics.tdist(A @ Ai, np.eye(4)))
print(metrics.tdist(Ai @ A, np.eye(4)))

[[ 0.55523701 -0.38569395  0.27965706  0.53797537]
 [-0.05875729  0.09195237  0.5776943  -0.02055776]
 [ 0.3610978  -0.68108284 -0.06321464 -0.14090037]
 [-1.0338259   0.40856018  0.03158238 -0.19588969]]
[[ 0.55523701 -0.38569395  0.27965706  0.53797537]
 [-0.05875729  0.09195237  0.5776943  -0.02055776]
 [ 0.3610978  -0.68108284 -0.06321464 -0.14090037]
 [-1.0338259   0.40856018  0.03158238 -0.19588969]]
4.2966547712338835e-16
5.390431360858285e-16
6.507066344842252e-16
